In [1]:
import cv2
from __future__ import division, print_function
# given an image and a list of points, we want to extract patches s.t. they are centered (vertically) on the tail edge
# we also want to extract negative patches, but this is a different task
def extract(img, center, size):
    # assume center is in x, y format
    upper_border = center[1] - size[0]//2
    lower_border = center[1] + size[0]//2
    left_border = center[0] - size[1]//2
    right_border = center[0] + size[1]//2
    
    extracted_patch = img[max(0,upper_border):min(img.shape[0],lower_border),
                          max(0,left_border):min(img.shape[1],right_border)]
    
    fillrow_amount = 0
    if upper_border < 0:
        fillrow_amount = upper_border
    elif lower_border > img.shape[0]: # assume that the patch's y isn't > img's y
        fillrow_amount = lower_border - img.shape[0]
    
    # fill the row before we fill the col if we need to
    if fillrow_amount != 0:
        tovstack = [extracted_patch]
        if fillrow_amount < 0:
            tovstack.insert(0,[extracted_patch[0,:]]*abs(fillrow_amount))
        else:
            tovstack.append([extracted_patch[-1,:]]*fillrow_amount)
        extracted_patch = np.vstack(tovstack)
            
    fillcol_amount = None
    if left_border < 0:
        fillcol_amount = left_border
    elif right_border > img.shape[1]:
        fillcol_amount = right_border - img.shape[1]
    
    if fillcol_amount != 0:
        tohstack = [extracted_patch]
        if fillcol_amount < 0:
            tohstack.insert(0,[extracted_patch[:,0].reshape(-1,1)]*abs(fillcol_amount))
        else:
            tohstack.append([extracted_patch[:,-1].reshape(-1,1)]*fillcol_amount)
        extracted_patch = np.hstack(tohstack)
    
    assert(extracted_patch.shape == size)
    return extracted_patch, upper_border
    
def find_best_y(img, lower_real_coord):    
    centerx = img.shape[1] // 2
    gradient_y_image = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)
    new_y_center = np.argmax(gradient_y_image[:,centerx])
    return new_y_center

def extract_candidates(img, point_list, size):
    candidates = []
    for point in point_list:
        initial_candidate, upper_border = extract(img, size, point)
        new_y_center = upper_border + find_best_y(initial_candidate) # point is x,y
        candidates.append(extract(img, size, (point[0],new_y_center)))
    return candidates
